In [52]:
import numpy as np
import pandas as pd

import itertools
import math

import matplotlib.pyplot as plt
from matplotlib import gridspec

from tqdm.notebook import tqdm
import pymorphy2

import nltk
from nltk.tokenize import RegexpTokenizer

from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

from sklearn.feature_extraction.text import CountVectorizer

import tqdm

from pymystem3 import Mystem

from Levenshtein import distance as lev


%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
from notebook.notebookapp import NotebookApp

NotebookApp.iopub_data_rate_limit=100000000.0

Smol playground

In [54]:
mystem = Mystem()
text = 'Как насчёт небольшого стемминга'
lemmas = mystem.lemmatize(text)
print(''.join(lemmas))

как насчет небольшой стемминг



In [55]:
convert = {
    'A': 'ADJ',
    'ADV': 'ADV',
    'ADVPRO': 'ADV',
    'ANUM': 'ADJ',
    'APRO': 'DET',
    'COM': 'ADJ',
    'CONJ': 'SCONJ',
    'INTJ': 'INTJ',
    'NONLEX': 'X',
    'NUM': 'NUM',
    'PART': 'PART',
    'PR': 'ADP',
    'S': 'NOUN',
    'SPRO': 'PRON',
    'UNKN': 'X',
    'V': 'VERB'
}

In [56]:
def tag(word='пожар'):
    from pymystem3 import Mystem
    stemmer = Mystem()
    processed = stemmer.analyze(word)[0]
    lemma = processed["analysis"][0]["lex"].lower().strip()
    pos = processed["analysis"][0]["gr"].split(',')[0]
    pos = pos.split('=')[0].strip()
    pos = convert[pos]
    tagged = lemma + '_' + pos
    return lemma, pos
tag('какой')

('какой', 'DET')

In [57]:
import gensim.downloader as api

info = api.info()  # show info about available models/datasets
model = api.load("word2vec-ruscorpora-300")

nltk.download('stopwords')
# pymorphy2 анализатор
morph = pymorphy2.MorphAnalyzer()
# стоп слова из nltk
stops = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to /home/art-
[nltk_data]     bash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
lev

<cyfunction levenshtein_distance at 0x7f8502fe52b0>

In [59]:
baddest_words = []

In [61]:
entire_vocab = model.key_to_index

def get_top_closest_in_vocab_and_tag(word):
    results = [] # list of (dist, word)
    try:
        word = ''.join([c for c in word if c.isalpha()])
        n_word, pos = tag(word)
    except:
        baddest_words.append(word)
        return [[100000, ':(']], word, ''
    for c in entire_vocab:
        key, key_pos = c.split('_')
        dist = lev(key, n_word)
        results.append([dist, c])
    return sorted(results)[:10], n_word, pos


get_top_closest_in_vocab_and_tag('наличие')

([[0, 'наличие_NOUN'],
  [2, 'величие_NOUN'],
  [2, 'галичий_ADJ'],
  [2, 'галичин_NOUN'],
  [2, 'куличие_NOUN'],
  [2, 'наитие_NOUN'],
  [2, 'налимий_ADJ'],
  [2, 'налич_NOUN'],
  [2, 'наличка_NOUN'],
  [2, 'наличник_NOUN']],
 'наличие',
 'NOUN')

In [41]:
with open('../dat/glue_all_but_owner.txt', 'r') as fl:
    messages = fl.readlines()
    messages = [m.replace('\n', '') for m in messages]

rs = []

def normalise_text(txt):
    words = [''.join(c for c in x if c.isalpha()) for x in txt.split()]
    words = [x for x in [morph.normal_forms(word)[0] for word in words ]\
            if x not in stops]
    return ' '.join(words)

for msg in tqdm.tqdm(messages):
    words = [normalise_text(x) for x in msg.split()]
    for word in words:
        print(word)
        res = get_top_closest_in_vocab_and_tag(word)
        if res[0][0][0] > 0:
            rs.append(res[0])
    break

  0%|          | 0/31772 [00:00<?, ?it/s]

ассалам
алейкума

заказать
чёрный
долго
ждать

цвет

синий
ми

сие
сегодня

смотреть




сколько

гб




сколько
гб

дело
реза
хийл
понять



чёрный
цвет

вариант
мочь
дать
знать


хотеть

всё
баркал



гб
ассалам
алейкума
цена
ала
брат



скорый
время
баракалла

розовый
цвет




наличие

мм



скидка
мочь
предложить
брат
мочь


смочь
отдать
гулкх
дийра
дар
ахь


суббота
забрать
сестра
брат
ассалам
алейкума
скинуть

реквизит

перевести
деньга

часы
баракалла
перевести
прийти
сестрёнка
пусть
забрать
розовый
дать
беркат
дойла
массарна
хийл

мм
ассалам
алейкума
брат
кхи
цхь
часыш
дезш
хьать
ду


наличие
чёрный


цена
отдать
завтра
забрать
отложить


вариант


  0%|          | 0/31772 [01:44<?, ?it/s]

In [51]:
rs

[[[1, 'асса_NOUN'],
  [1, 'ассад_NOUN'],
  [1, 'ассам_NOUN'],
  [1, 'вассал_NOUN'],
  [2, 'авал_NOUN'],
  [2, 'аврал_NOUN'],
  [2, 'айхал_NOUN'],
  [2, 'аксай_NOUN'],
  [2, 'амбал_NOUN'],
  [2, 'анал_NOUN']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[1, 'хилл_NOUN'],
  [1, 'хойл_NOUN'],
  [2, 'аил_NOUN'],
  [2, 'ахилл_NOUN'],
  [2, 'ахиол_NOUN'],
  [2, 'бигл_NOUN'],
  [2, 'бий_NOUN'],
  [2, 'билл_NOUN'],
  [2, 'биол_NOUN'],
  [2, 'бойл_NOUN']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[100000, ':(']],
 [[1, 'вге_NOUN'],
  [1, 'ве_NOUN'],
  [1, 'вее_NOUN'],
  [1, 'вже_NOUN'],
  [1, 'вне_ADP'],
  [1, 'вне_ADV'],
  [1, 'вре_NOUN'],
  [1, 'вс_NOUN'],
  [1, 'всв_NOUN'],
  [1,

In [43]:
#raw_data = pd.read_csv('doroga_v_les.csv', error_bad_lines=False).fillna('')
#raw_data

In [44]:
#data = pd.DataFrame(raw_data.apply(''.join, axis=1))
#data.columns = ['text']
#data.to_csv('doroga_v_les.csv')

In [45]:
data = pd.read_csv('../dat/glue_split_owner.txt', sep=';', header=None)
data.columns = ['text']

In [46]:
nltk.download('punkt')
sent_tokenizer = lambda sent: nltk.sent_tokenize(sent, language="russian")

[nltk_data] Downloading package punkt to /home/art-bash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
nltk.download('stopwords')
# pymorphy2 анализатор
morph = pymorphy2.MorphAnalyzer()
# стоп слова из nltk
stops = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to /home/art-
[nltk_data]     bash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
def proccess_text(text):
    text = text.lower()
    sents = sent_tokenizer(text)
    words = list(
        itertools.chain.from_iterable(
            word_tokenizer.tokenize_sents(sents)))
    return [x for x in [morph.normal_forms(word)[0] for word in words ]\
            if x not in stops]

In [49]:
word_tokenizer = RegexpTokenizer(r'[a-zа-яёЁА-ЯA-Z]\w+\'?\w*')

data_text = data['text']

for i in tqdm.tqdm(range(len(data))):
     data_text.iloc[i] = ' '.join(proccess_text(data_text.iloc[i]))
        
data['text'] = data_text
#data_text.columns = ['text']
data.to_csv('doroga_v_les_proccessed.csv')
data

 32%|███▏      | 20307/63860 [00:33<01:12, 602.36it/s]


KeyboardInterrupt: 

In [ ]:
#data_text = pd.read_csv('doroga_v_les_proccessed.csv', sep=';')
#data_text

## Filter

In [ ]:
#data_text = pd.DataFrame(data_text.apply(replace('добрый', ''), axis=1))
#data_text['text'] = data_text['text'].str.replace('добрый','')
#data_text['text'] = data_text['text'].str.replace('день','')
#data_text['text'] = data_text['text'].str.replace('здравствуйте','')
#data_text['text'] = data_text['text'].str.replace('утро','')
#data_text['text'] = data_text['text'].str.replace('спасибо','')

#data

In [ ]:
tokenizer = RegexpTokenizer(r'[a-zа-яёЁА-ЯA-Z]\w+\'?\w*')
data['tokenized_text'] = data['text'].apply(lambda row: tokenizer.tokenize(row))

data

# Tutorial

In [ ]:
data['tokenized_text']

## Filter greetings

In [ ]:
def get_most_freq_words(str, n=None):
    vect = CountVectorizer().fit(str)
    bag_of_words = vect.transform(str)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq =sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]

get_most_freq_words([word for messege in data.tokenized_text for word in messege],10)

In [ ]:
messeges_dictionary = Dictionary(data.tokenized_text)

messeges_corpus = [messeges_dictionary.doc2bow(messege) for messege in data.tokenized_text]

# compute coherence
messeges_coherence = []
for nb_topics in tqdm.tqdm(range(1,36)):
    lda = LdaModel(messeges_corpus, num_topics = nb_topics, id2word = messeges_dictionary, passes=10)
    cohm = CoherenceModel(model=lda, corpus=messeges_corpus, dictionary=messeges_dictionary, coherence='u_mass')
    coh = cohm.get_coherence()
    messeges_coherence.append(coh)

# visualize coherence
plt.figure(figsize=(10,5))
plt.plot(range(1,36),messeges_coherence)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score");

In [ ]:
k = 30 # number of topics

messeges_lda = LdaModel(messeges_corpus, num_topics = k, id2word = messeges_dictionary, passes=10)

def plot_top_words(lda=messeges_lda, nb_topics=k, nb_words=15):
    top_words = [[word for word,_ in lda.show_topic(topic_id, topn=50)] for topic_id in range(lda.num_topics)]
    top_betas = [[beta for _,beta in lda.show_topic(topic_id, topn=50)] for topic_id in range(lda.num_topics)]

    gs  = gridspec.GridSpec(round(math.sqrt(k))+1,round(math.sqrt(k))+1)
    gs.update(wspace=0.5, hspace=0.5)
    plt.figure(figsize=(20,15))
    for i in range(nb_topics):
        ax = plt.subplot(gs[i])
        plt.barh(range(nb_words), top_betas[i][:nb_words], align='center',color='blue', ecolor='black')
        ax.invert_yaxis()
        ax.set_yticks(range(nb_words))
        ax.set_yticklabels(top_words[i][:nb_words])
        plt.title("Topic "+str(i))
        
plot_top_words()

## Hrabovoy

In [ ]:
from sklearn.utils import shuffle

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
texts = data['text'].values.tolist()
topics = data['tag'].values.tolist()

texts, topics = shuffle(texts, topics)

texts_train = texts[:45]
texts_test = texts[45:]

topics_train = topics[:45]
topics_test = topics[45:]

In [ ]:
vectorizer = CountVectorizer(binary=True)
vectorizer.fit(texts_train)

In [ ]:
train_doc_vectors = vectorizer.transform(texts_train)

In [ ]:
lda = LatentDirichletAllocation(n_components=4, random_state=42)
lda.fit(train_doc_vectors)

In [ ]:
feature_train = lda.transform(train_doc_vectors)
topics_train = np.array(topics_train)

In [ ]:
classifier = RandomForestClassifier(n_estimators=10)
classifier.fit(feature_train, topics_train)

In [ ]:
test_doc_vectors = vectorizer.transform(texts_test)
feature_test = lda.transform(test_doc_vectors)

In [ ]:
pred = classifier.predict(feature_test)
pred

In [ ]:
topics_test = np.array(topics_test)
topics_test

In [ ]:
print(classification_report(topics_test, pred, zero_division=1))